### Import the libraries , read data and perform EDA. 



In [276]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
from sklearn.impute import SimpleImputer
from surprise import SVD
from surprise.dataset import Dataset
from surprise.reader import Reader
from scipy.sparse.linalg import svds
from surprise.model_selection import train_test_split,cross_validate
from surprise.model_selection.split import KFold
from surprise.prediction_algorithms.knns import KNNWithMeans
from surprise import accuracy


In [4]:
phone1=pd.read_csv('phone_user_review_file_1.csv',encoding='latin1')
phone2=pd.read_csv('phone_user_review_file_2.csv',encoding='latin1')
phone3=pd.read_csv('phone_user_review_file_3.csv',encoding='latin1')
phone4=pd.read_csv('phone_user_review_file_4.csv',encoding='latin1')
phone5=pd.read_csv('phone_user_review_file_5.csv',encoding='latin1')

In [5]:
#Merging all the files as one dataframe
frames = [phone1, phone2, phone3,phone4,phone5]
phoneReview_df=pd.concat(frames)

In [6]:
#Shape of the DF
phoneReview_df.shape

(1251296, 11)

In [7]:
#Datatypes
phoneReview_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1251296 entries, 0 to 350215
Data columns (total 11 columns):
 #   Column     Non-Null Count    Dtype  
---  ------     --------------    -----  
 0   phone_url  1251296 non-null  object 
 1   date       1251296 non-null  object 
 2   lang       1251296 non-null  object 
 3   country    1251296 non-null  object 
 4   source     1251296 non-null  object 
 5   domain     1251296 non-null  object 
 6   score      1199479 non-null  float64
 7   score_max  1199479 non-null  float64
 8   extract    1234823 non-null  object 
 9   author     1201151 non-null  object 
 10  product    1251295 non-null  object 
dtypes: float64(2), object(9)
memory usage: 114.6+ MB


In [8]:
#5 point summary
phoneReview_df.describe()

,score,score_max
count,1.199479e+06,1199479.0
mean,8.050337e+00,10.0
std,2.622776e+00,0.0
min,2.000000e-01,10.0
25%,8.000000e+00,10.0
50%,1.000000e+01,10.0
75%,1.000000e+01,10.0
max,1.000000e+01,10.0


In [9]:
#Check few observations
phoneReview_df.head(5)

,phone_url,date,lang,country,source,domain,score,score_max,extract,author,product
0,/cellphones/samsung-galaxy-s8/,5/2/2017,en,us,Verizon Wireless,verizonwireless.com,10.0,10.0,As a diehard Samsung fan who has had every Sam...,CarolAnn35,Samsung Galaxy S8
1,/cellphones/samsung-galaxy-s8/,4/28/2017,en,us,Phone Arena,phonearena.com,10.0,10.0,Love the phone. the phone is sleek and smooth ...,james0923,Samsung Galaxy S8
2,/cellphones/samsung-galaxy-s8/,5/4/2017,en,us,Amazon,amazon.com,6.0,10.0,Adequate feel. Nice heft. Processor's still sl...,R. Craig,"Samsung Galaxy S8 (64GB) G950U 5.8"" 4G LTE Unl..."
3,/cellphones/samsung-galaxy-s8/,5/2/2017,en,us,Samsung,samsung.com,9.2,10.0,Never disappointed. One of the reasons I've be...,Buster2020,Samsung Galaxy S8 64GB (AT&T)
4,/cellphones/samsung-galaxy-s8/,5/11/2017,en,us,Verizon Wireless,verizonwireless.com,4.0,10.0,I've now found that i'm in a group of people t...,S Ate Mine,Samsung Galaxy S8


In [10]:
#Round off Score column values
phoneReview_df = phoneReview_df.round({'score': 0})


In [11]:
phoneReview_df

,phone_url,date,lang,country,source,domain,score,score_max,extract,author,product
0,/cellphones/samsung-galaxy-s8/,5/2/2017,en,us,Verizon Wireless,verizonwireless.com,10.0,10.0,As a diehard Samsung fan who has had every Sam...,CarolAnn35,Samsung Galaxy S8
1,/cellphones/samsung-galaxy-s8/,4/28/2017,en,us,Phone Arena,phonearena.com,10.0,10.0,Love the phone. the phone is sleek and smooth ...,james0923,Samsung Galaxy S8
2,/cellphones/samsung-galaxy-s8/,5/4/2017,en,us,Amazon,amazon.com,6.0,10.0,Adequate feel. Nice heft. Processor's still sl...,R. Craig,"Samsung Galaxy S8 (64GB) G950U 5.8"" 4G LTE Unl..."
3,/cellphones/samsung-galaxy-s8/,5/2/2017,en,us,Samsung,samsung.com,9.0,10.0,Never disappointed. One of the reasons I've be...,Buster2020,Samsung Galaxy S8 64GB (AT&T)
4,/cellphones/samsung-galaxy-s8/,5/11/2017,en,us,Verizon Wireless,verizonwireless.com,4.0,10.0,I've now found that i'm in a group of people t...,S Ate Mine,Samsung Galaxy S8
...,...,...,...,...,...,...,...,...,...,...,...
350211,/cellphones/sony-ericsson-z710i/,8/7/2006,fr,fr,GraphMobile,graphmobile.com,10.0,10.0,Pour info il est sur amazon.de a 212.99â¬ s'i...,NaN,Sony-Ericsson Z710i
350212,/cellphones/sony-ericsson-z710i/,8/5/2006,fr,fr,GraphMobile,graphmobile.com,9.0,10.0,HabituÃ© Ã samsung sony nous sort 1 jolie cla...,NaN,Sony-Ericsson Z710i
350213,/cellphones/sony-ericsson-z710i/,7/19/2006,fr,fr,GraphMobile,graphmobile.com,10.0,10.0,"Pour les gens qui ne regarde pas Il fait mp3, ...",NaN,Sony-Ericsson Z710i
350214,/cellphones/sony-ericsson-z710i/,7/9/2006,fr,fr,GraphMobile,graphmobile.com,9.0,10.0,C vrai que sans le mp3 c moyen...,NaN,Sony-Ericsson Z710i


In [12]:
#Check for missing values
phoneReview_df.isnull().sum()

phone_url        0
date             0
lang             0
country          0
source           0
domain           0
score        51817
score_max    51817
extract      16473
author       50145
product          1
dtype: int64

In [13]:
#Here we can see that we have 2 int columns and 3 object type columns  that have missing values
#Handling Int columns by replacing missing values with the median of respective columns
imputeValues = ['score','score_max']
impute =SimpleImputer(missing_values=np.NaN,strategy ='median')
impute.fit(phoneReview_df[imputeValues])
phoneReview_df[imputeValues]=impute.transform(phoneReview_df[imputeValues])

In [14]:
#Dropping the null values rowwise from categorical features

phoneReview_df.dropna(axis=0,inplace=True)

In [15]:
#missing values post handling
phoneReview_df.isnull().sum()

phone_url    0
date         0
lang         0
country      0
source       0
domain       0
score        0
score_max    0
extract      0
author       0
product      0
dtype: int64

In [16]:
phoneReview_df

,phone_url,date,lang,country,source,domain,score,score_max,extract,author,product
0,/cellphones/samsung-galaxy-s8/,5/2/2017,en,us,Verizon Wireless,verizonwireless.com,10.0,10.0,As a diehard Samsung fan who has had every Sam...,CarolAnn35,Samsung Galaxy S8
1,/cellphones/samsung-galaxy-s8/,4/28/2017,en,us,Phone Arena,phonearena.com,10.0,10.0,Love the phone. the phone is sleek and smooth ...,james0923,Samsung Galaxy S8
2,/cellphones/samsung-galaxy-s8/,5/4/2017,en,us,Amazon,amazon.com,6.0,10.0,Adequate feel. Nice heft. Processor's still sl...,R. Craig,"Samsung Galaxy S8 (64GB) G950U 5.8"" 4G LTE Unl..."
3,/cellphones/samsung-galaxy-s8/,5/2/2017,en,us,Samsung,samsung.com,9.0,10.0,Never disappointed. One of the reasons I've be...,Buster2020,Samsung Galaxy S8 64GB (AT&T)
4,/cellphones/samsung-galaxy-s8/,5/11/2017,en,us,Verizon Wireless,verizonwireless.com,4.0,10.0,I've now found that i'm in a group of people t...,S Ate Mine,Samsung Galaxy S8
...,...,...,...,...,...,...,...,...,...,...,...
350201,/cellphones/sony-ericsson-z710i/,12/28/2006,de,de,Guenstiger.de,guenstiger.de,10.0,10.0,Das Handy ist spitze ! Das Menue erklÃ¤rt sich...,Sven P.,Sony-Ericsson Z710i
350202,/cellphones/sony-ericsson-z710i/,12/28/2006,de,de,Guenstiger.de,guenstiger.de,10.0,10.0,Das Handy habe ich schon ca. vier Wochen und b...,Moran,Sony-Ericsson Z710i
350203,/cellphones/sony-ericsson-z710i/,12/20/2006,de,de,Inside-handy,inside-handy.de,8.0,10.0,"Mein Z710i in Schwarz Optisch ansprechend, Akk...",Torsten,Sony Ericsson Z710i
350206,/cellphones/sony-ericsson-z710i/,11/25/2006,fr,fr,GraphMobile,graphmobile.com,10.0,10.0,Le W710i c'est la mÃªme chose avec: -Ã©couteur...,Zell,Sony-Ericsson Z710i


In [17]:
#Drop duplicates
phoneReview_df.drop_duplicates(inplace=True)

In [18]:
phoneReview_df.shape

(1183377, 11)

#### Around 4477 rows have been dropped due to duplicates.

#### Keep only 1000000 data samples. Use random state=612.


In [19]:
phoneReview_df.nunique()

phone_url       3189
date            5738
lang              21
country           40
source           296
domain           345
score             11
score_max          1
extract      1124056
author        698342
product        51292
dtype: int64

In [20]:

phoneReview_sample = phoneReview_df.sample(n=1000000,random_state=612)
phoneReview_sample.shape

(1000000, 11)

In [21]:
#Dropping irrelevant features
drop_columns =['phone_url','date','lang','country','domain']
phoneReview_sample.drop(columns=drop_columns,inplace=True)

In [22]:
phoneReview_sample

,source,score,score_max,extract,author,product
150329,Amazon,10.0,10.0,Uno smartphone di fascia media di tutto rispet...,CLIENTE AMAZON LP84,"Honor 6X Smartphone 4G LTE, Diplay 5.5 pollici..."
258061,Amazon,10.0,10.0,Very nice Mobile good quality for camera,karthick,"Asus Zenfone 2 Laser ZE500KL (Black, 16GB)"
110457,Amazon,10.0,10.0,very good for the price paid has everything i ...,steve pearce,Apple iPhone 5c 16GB White SIM-Free Smartphone...
7936,Amazon,6.0,10.0,Count me as one of the many with charging port...,Captain Ron,"BLU Vivo XL Smartphone - 5.5"" 4G LTE - GSM Unl..."
144185,RueDuCommerce,10.0,10.0,LE TOP,Dominique,LG Optimus G Pro Noir
...,...,...,...,...,...,...
102667,Amazon,2.0,10.0,Don't buy any samsung mobiles.after purchase t...,NIRAJ TIWARI,"Samsung Galaxy S6 (Blue Topaz, 32GB)"
165201,Amazon,10.0,10.0,"My wife fell in love with this phone, which ha...",Lawrence E. Butler,Nokia C3 (C3-00) WHITE Unlocked Phone
210117,Amazon,4.0,10.0,"OK, but when a message arrives, no matter what...",Amazon Customer,"Nokia 6700 slide, mobile phone aluminium EU un..."
189681,Ciao,8.0,10.0,Bueno aquÃÂ­ os traigo un gran terminal el Do...,ginesaka,Doogee Voyager DG300


### Analysis of users/products ratings

In [23]:
print("Count of unique products :",phoneReview_sample['product'].nunique())
print("Count of unique users :",phoneReview_sample['author'].nunique())

Count of unique products : 49199
Count of unique users : 604386


#### Identify the most rated features.

In [452]:

# We will count the products that have been rated by many users. This won't consider whether it was rated nicely or poorly.

ratingsCount = pd.DataFrame(phoneReview_sample.groupby('product')['score'].count())
ratingsCount['rating_counts'] = pd.DataFrame(phoneReview_sample.groupby('product')['score'].count())  
ratingsCount=ratingsCount.sort_values(by='rating_counts',ascending=False)

In [453]:
ratingsCount

,score,rating_counts
product,,
"Lenovo Vibe K4 Note (White,16GB)",4382,4382
"Lenovo Vibe K4 Note (Black, 16GB)",3671,3671
"OnePlus 3 (Graphite, 64 GB)",3527,3527
"OnePlus 3 (Soft Gold, 64 GB)",3000,3000
Huawei P8lite zwart / 16 GB,2269,2269
...,...,...
Nokia C5-03 (Lime green),1,1
"Nokia C5-00, NAVI, TÃ©lÃ©phone portable warm gris 10 Tracks kostenlos dÃ©bloquÃ©, logiciel original o",1,1
Nokia C5-00 Noir Symbian OS 9.3 / Series 60 3rd Edition Feature Pack 2,1,1


In [454]:
#We will consider the items that have been rated more than 1000 times
ratingsCount = ratingsCount[ratingsCount['rating_counts']>1000]
print("Products that have been most rated : \n",ratingsCount)


Products that have been most rated : 
                                                     score  rating_counts
product                                                                 
Lenovo Vibe K4 Note (White,16GB)                     4382           4382
Lenovo Vibe K4 Note (Black, 16GB)                    3671           3671
OnePlus 3 (Graphite, 64 GB)                          3527           3527
OnePlus 3 (Soft Gold, 64 GB)                         3000           3000
Huawei P8lite zwart / 16 GB                          2269           2269
Samsung Galaxy Express I8730                         2260           2260
Lenovo Vibe K5 (Gold, VoLTE update)                  2114           2114
Samsung Galaxy S6 zwart / 32 GB                      1938           1938
Lenovo Vibe K5 (Grey, VoLTE update)                  1757           1757
Lenovo Used Lenovo Zuk Z1 (Space Grey, 64GB)         1639           1639
OnePlus 3T (Gunmetal, 6GB RAM + 64GB memory)         1603           1603
Samsung Gala

In [436]:
print("Count of products that have been most rated : ",ratingsCount.shape[0])

Count of products that have been most rated :  40


#### Identify the users with most number of reviews.

In [394]:

print("Users with most number of reviews :\n",phoneReview_sample.groupby('author')['score'].count().sort_values(ascending=False).head(10))


Users with most number of reviews :
 author
Amazon Customer    64264
Cliente Amazon     16334
e-bit               7091
Client d'Amazon     6454
Amazon Kunde        3939
einer Kundin        2211
einem Kunden        1608
Anonymous           1506
unknown             1455
Anonymous           1229
Name: score, dtype: int64


In [25]:
#Verify if above author names exist
phoneReview_sample[phoneReview_sample.author=='Amazon Customer']

,source,score,score_max,extract,author,product
346655,Amazon,8.0,10.0,The phone was for use in the Caribbean and it ...,Amazon Customer,Samsung Galaxy S5 SM-G900A GSM Unlocked Cellph...
112955,Amazon,8.0,10.0,If people compromise with Expandable memory an...,Amazon Customer,"Mi 4i (White, 16GB)"
52435,Amazon,10.0,10.0,"Phone is very fine, but Phone tips & guideline...",Amazon Customer,"Lenovo Vibe K4 Note (Black, 16GB)"
49315,Amazon,8.0,10.0,Phone lives up to it's sales hype. Decent phon...,Amazon Customer,"Lenovo Vibe K4 Note (Black, 16GB)"
297330,Amazon,8.0,10.0,Nice product ....! Without waste of time me an...,Amazon Customer,"Motorola Moto G5 (3 GB, Lunar Grey)"
...,...,...,...,...,...,...
70453,Amazon,8.0,10.0,Yu yuphoria is best for procesing and camera c...,Amazon Customer,YU Yuphoria YU5010A (Black+Silver)
115777,Amazon,2.0,10.0,Had no idea the phone was so small. I had prob...,Amazon Customer,BLU Dash JR D141W Dual Sim Factory Unlocked An...
213450,Amazon,2.0,10.0,very bad quality product..,Amazon Customer,Nokia 230 Dual Sim (Dark Silver)
23566,Amazon,2.0,10.0,The software is very bad.It gives a lot of pro...,Amazon Customer,"Lenovo Vibe K5 (Grey, VoLTE update)"


#### Identifying products having more than 50 ratings and users who have given more than 50 ratings


#### Products having more than 50 ratings

In [26]:
product_count= pd.DataFrame(phoneReview_sample.groupby('product')['score'].count().sort_values(ascending=False))
product_count=product_count[product_count['score']>50]
product_count

,score
product,
"Lenovo Vibe K4 Note (White,16GB)",4382
"Lenovo Vibe K4 Note (Black, 16GB)",3671
"OnePlus 3 (Graphite, 64 GB)",3527
"OnePlus 3 (Soft Gold, 64 GB)",3000
Huawei P8lite zwart / 16 GB,2269
...,...
"Motorola Defy + MB526 Unlocked GSM Phone with Android 2.3 OS, Touchscreen, MOTOBLUR , 5MP Camera, GPS, Wi-Fi, Bluetooth, 3G 850/2100MHZ , FM Radio and microSD Slot - Graphite Grey",51
APPLE iPHONE 3G 8 GB CEP TELEFONU,51
CUBOT S208 5.0 Inch IPS Screen MTK6582M 1.3GHz 1GB/16GB Android 4.2 Dual SIM Wifi Bluetooth Smartphone (Black+...,51


#### Users who have provided more than 50 ratings

In [27]:
user_count= phoneReview_sample.groupby('author')['score'].count().sort_values(ascending=False)
user_count=user_count[user_count>50]
user_count

author
Amazon Customer    64264
Cliente Amazon     16334
e-bit               7091
Client d'Amazon     6454
Amazon Kunde        3939
                   ...  
Rohit                 51
Stefania              51
Samantha              51
bob                   51
Sumit                 51
Name: score, Length: 709, dtype: int64

In [28]:
#print the shape of results
print("Count of users who have provided more than 50 ratings: ",user_count.shape[0])
print("Count of products that have received more than 50 ratings :",product_count.shape[0])

Count of users who have provided more than 50 ratings:  709
Count of products that have received more than 50 ratings : 4586


### Recommend top 5 mobile phones using popularity based model
#### In this exercise, we will first take the highest mean scores of all products. We will then count how many users have rated the product with the mean score.

In [413]:
#Lets check the scale of rating and verify how many products have received the max scale
scale_rating =phoneReview_sample['score'].unique()

In [414]:
print("The rating scale ranges from {} to {} lowest to highest".format(scale_rating.min(),scale_rating.max()))

The rating scale ranges from 0.0 to 10.0 lowest to highest


In [415]:
#We will take mean of scores of all products in the sampled dataset and count how many have users have rated the mean score

ratings_mean_count = pd.DataFrame(phoneReview_sample.groupby('product')['score'].mean())
ratings_mean_count['rating_counts'] = pd.DataFrame(phoneReview_sample.groupby('product')['score'].count())  
ratings_mean_count=ratings_mean_count.sort_values(by='score',ascending=False)

In [416]:
# we will check  how many users have rated the products with the highest score i.e 10.0
ratings_mean_count=ratings_mean_count[ratings_mean_count['score']==10.0]
ratings_mean_count

,score,rating_counts
product,,
è·¨å¹´åå ä¸èµ·ä¾ç Nokia C5-03 ç«éè©¦ç¨å ±å,10.0,1
Smartphone Apple iPhone 5C 16GB,10.0,3
Smartphone Apple iPhone 5 32GB Libre,10.0,1
Smartphone Apple iPhone 5 32GB,10.0,1
Huawei Y6 8GB,10.0,1
...,...,...
Alcatel One Touch POP S3 5050Y white,10.0,1
Ð¡Ð¾ÑÐ¾Ð²ÑÐ¹ ÑÐµÐ»ÐµÑÐ¾Ð½ HTC One X+ 64 Gb Black,10.0,9
Sony-Ericsson Xperia active,10.0,5


In [417]:
ratings_mean_count.tail(5)

,score,rating_counts
product,,
Alcatel One Touch POP S3 5050Y white,10.0,1
Ð¡Ð¾ÑÐ¾Ð²ÑÐ¹ ÑÐµÐ»ÐµÑÐ¾Ð½ HTC One X+ 64 Gb Black,10.0,9
Sony-Ericsson Xperia active,10.0,5
Ð¡Ð¾ÑÐ¾Ð²ÑÐ¹ ÑÐµÐ»ÐµÑÐ¾Ð½ HTC One V Grey,10.0,8
Ð¡Ð¾ÑÐ¾Ð²ÑÐ¹ ÑÐµÐ»ÐµÑÐ¾Ð½ HTC One V Black,10.0,10


In [418]:
#range of count of users for products having 10 as mean score
ratings_mean_count['rating_counts'].unique()

array([  1,   3,   4,   5,   2,   7,  40,  38,  41,  43,  14,   9,  13,
        10,  12,  20,   8,  11,   6,  24,  15,  18,  22,  19,  21,  17,
        23,  26,  16,  42,  39,  45,  44,  35,  37,  36,  46,  29,  32,
        31,  25,  30,  28,  51,  59,  63,  47,  49,  27,  58,  52, 114,
        68,  70,  87, 134, 142, 130,  69,  62, 212,  34,  50, 103,  54,
        33,  86, 145,  55, 104,  83, 139,  56,  66, 260, 111,  57,  64,
       183, 153, 154,  53,  61, 550,  71,  67, 248, 353, 210, 106,  79,
        88, 101,  65,  72, 311, 102,  90,  81, 113, 120, 143, 158, 128,
       169, 126,  48, 170, 286, 157,  91,  92,  97, 233], dtype=int64)

In [419]:
#We will take 150 users as threshold value to count as popularly and highly rated product 
ratings_mean_count=ratings_mean_count[ratings_mean_count['rating_counts']>150]
ratings_mean_count

,score,rating_counts
product,,
HTC DESIRE HD,10.0,212
Nokia 5800 XpressMusic Cep Telefonu,10.0,260
SONY ERICSSON Vivaz,10.0,183
SONY ERICSSON Xperia X10,10.0,153
SAMSUNG Galaxy SII (S2),10.0,154
SAMSUNG Player Star 2,10.0,550
Samsung (936) Galaxy Ace S5830I,10.0,248
SAMSUNG Galaxy S,10.0,353
SAMSUNG Galaxy Ace,10.0,210


In [420]:
ratings_products=pd.DataFrame(ratings_mean_count.sort_values(by='rating_counts',ascending=False).head(5))
ratings_products

,score,rating_counts
product,,
SAMSUNG Player Star 2,10.0,550
SAMSUNG Galaxy S,10.0,353
NOKIA E71 Noir,10.0,311
Vodafone (69) 858 Smart,10.0,286
Nokia 5800 XpressMusic Cep Telefonu,10.0,260


In [421]:
#top 5 products
print("Top 5  recommended products based on popularity :\n",ratings_products.iloc[:,0])

Top 5  recommended products based on popularity :
 product
SAMSUNG Player Star 2                  10.0
SAMSUNG Galaxy S                       10.0
NOKIA E71 Noir                         10.0
Vodafone (69) 858 Smart                10.0
Nokia 5800 XpressMusic Cep Telefonu    10.0
Name: score, dtype: float64


In [37]:
#Cross-check if the count  matches with the original count 
samsung_score =phoneReview_sample[phoneReview_sample['product']=='SAMSUNG Player Star 2']
samsung_score= samsung_score[samsung_score['score']==10.0]
samsung_score.shape

(550, 6)

### Build collaborative filtering models using SVD and KNNWithMeans (item based and user based)



In [39]:
#Take 5000 samples and retrieve author (User),product(Item) and score (rating)
phoneReviewSampleSVD = phoneReview_df.sample(n=5000)
reader=Reader(rating_scale=(0,10))
#phoneReview_surprise = Dataset.load_from_df(phoneReview_df[['author','product','score']],reader)
phoneReview_SVD =phoneReviewSampleSVD[['author','product','score']]
phoneReview_SVD

,author,product,score
278070,Salvatori Bassano,"Lenovo ZUK Z1 4G Smartphone 5.5"" Fingerprint I...",10.0
269870,T. Vill,"Nokia E75 Smartphone (UMTS, GPS, UKW Radio, 3 ...",10.0
318920,bigvoron,Samsung I9100 Galaxy S II (Black),8.0
170036,Elizabeth McIntosh,Apple iPhone 3Gs 16GB black,2.0
179655,Loes,Huawei P8lite zwart / 16 GB,8.0
...,...,...,...
85639,MICHAEL MARISCAL,Sprint LG Volt White (Sprint Prepaid),6.0
198745,Lourdes,Samsung Galaxy A5 - Smartphone libre Android (...,10.0
238807,ÐÐ»ÐµÐºÑÐµÐ¹,Samsung I8190 Galaxy S III mini 8GB (ÑÐµÑÐ½Ñ...,9.0
82401,Jhennifer alves dos santos figueira,Smartphone Samsung Galaxy S6 Edge 64GB G925I D...,10.0


In [41]:
#Map each unique product Id and author Id to unique values
#Map product Ids
phoneReviewProdName=pd.DataFrame()
phoneReviewProdName['prodNames']=phoneReview_df['product'].unique()
phoneReviewProdName['prodId']=phoneReviewProdName.index

In [42]:
phoneReviewProdName

,prodNames,prodId
0,Samsung Galaxy S8,0
1,"Samsung Galaxy S8 (64GB) G950U 5.8"" 4G LTE Unl...",1
2,Samsung Galaxy S8 64GB (AT&T),2
3,"Samsung Galaxy S8, 64Gb - Black",3
4,"Samsung Galaxy S8, 64Gb - Orchid Grey",4
...,...,...
51287,Sony Ericsson Z710,51287
51288,Sony Ericsson Z710i,51288
51289,Sony Ericsson z710i,51289
51290,SONY ERICSSON Z710i CEP TELEFONU,51290


In [43]:
prodDict = phoneReviewProdName.set_index('prodNames').T.to_dict('list')
phoneReviewSampleSVD['#prodID']=phoneReviewSampleSVD['product'] .apply(lambda x:int(''.join(map(str, prodDict.get(x)))))

#phoneReview_sample['#prodID'] = phoneReview_sample['product'] .apply(lambda x: prodDict.get(x))

In [44]:
#Map Author Ids
authorReviewNames=pd.DataFrame()
authorReviewNames['authorName']=phoneReview_df['author'].unique()
authorReviewNames['authorId']=authorReviewNames.index


In [45]:
authorReviewNames

,authorName,authorId
0,CarolAnn35,0
1,james0923,1
2,R. Craig,2
3,Buster2020,3
4,S Ate Mine,4
...,...,...
698337,kradlegno,698337
698338,Schrott,698338
698339,oli02,698339
698340,Zell,698340


In [46]:
authorDict = authorReviewNames.set_index('authorName').T.to_dict('list')


In [47]:
authorDict

{'CarolAnn35': [0],
 'james0923': [1],
 'R. Craig': [2],
 'Buster2020 ': [3],
 'S Ate Mine': [4],
 'BDB76': [5],
 'KLC30306': [6],
 'Mnhy': [7],
 'Mr Alan': [8],
 'Raven Wolf Song': [9],
 'Settoloki': [10],
 'Karren': [11],
 'zenkitty': [12],
 'AT and T wins': [13],
 'Dwakal': [14],
 'fflinty': [15],
 'Desiree42': [16],
 'SassyRose': [17],
 'Robhig': [18],
 'JPinCJ': [19],
 'SquareBH': [20],
 'Sweetteri101': [21],
 'Timmy2toxic': [22],
 'grout': [23],
 'Agne': [24],
 'jshcox': [25],
 'Mp12': [26],
 'Hexxen': [27],
 'Cmoljak': [28],
 'tammyjohn': [29],
 'Lovely Wan': [30],
 'Ms_ Brandi': [31],
 'longo': [32],
 'joffre': [33],
 'RolloT': [34],
 'OlyO': [35],
 'dmc1355': [36],
 'Dkstl2017': [37],
 'E of a': [38],
 'luis1': [39],
 'Gamp': [40],
 'Stl Scott': [41],
 'boz2181': [42],
 'Gicook': [43],
 'Lucca': [44],
 'Kayelynne091w': [45],
 'Nynvolt': [46],
 'billd500': [47],
 'Shellsb': [48],
 'bobbydean': [49],
 'Jenee': [50],
 'Triciab82': [51],
 'Julz75': [52],
 'Charleyellajones666': [5

In [48]:
phoneReviewSampleSVD['#authorId']=phoneReviewSampleSVD['author'] .apply(lambda x:int(''.join(map(str, authorDict.get(x)))))


In [49]:
phoneReviewSampleSVD

,phone_url,date,lang,country,source,domain,score,score_max,extract,author,product,#prodID,#authorId
278070,/cellphones/lenovo-zuk-z1/,1/25/2016,it,it,Amazon,amazon.it,10.0,10.0,Grande telefono .. nulla da invidiare ad appar...,Salvatori Bassano,"Lenovo ZUK Z1 4G Smartphone 5.5"" Fingerprint I...",30890,425402
269870,/cellphones/nokia-e75/,9/1/2009,de,de,Amazon,amazon.de,10.0,10.0,Nachdem hier schon vieles abgehandelt wurde la...,T. Vill,"Nokia E75 Smartphone (UMTS, GPS, UKW Radio, 3 ...",47824,652462
318920,/cellphones/samsung-galaxy-s-ii/,11/25/2011,ru,ua,Hotline.ua,hotline.ua,8.0,10.0,Ð¡Ð¸Ð»ÑÐ½Ð¾ ÑÐ°ÑÑÑÑÐ¾Ð¸Ð»Ð° Ð²Ð¸Ð´ÐµÐ¾Ñ...,bigvoron,Samsung I9100 Galaxy S II (Black),49954,680724
170036,/cellphones/apple-iphone-3gs/,11/1/2013,en,gb,Amazon,amazon.co.uk,2.0,10.0,do not buy from this company - I placed an ord...,Elizabeth McIntosh,Apple iPhone 3Gs 16GB black,43829,593301
179655,/cellphones/huawei-p8-lite/,1/23/2016,nl,nl,KIESKEURIG,kieskeurig.nl,8.0,10.0,Heb de telefoon nog maar een paar dagen in bez...,Loes,Huawei P8lite zwart / 16 GB,6064,40164
...,...,...,...,...,...,...,...,...,...,...,...,...,...
85639,/cellphones/lg-volt-ls740/,5/25/2015,en,us,Amazon,amazon.com,6.0,10.0,was a wal-mart phone. Icould have gone there a...,MICHAEL MARISCAL,Sprint LG Volt White (Sprint Prepaid),21636,316954
198745,/cellphones/samsung-galaxy-a5/,3/14/2015,es,es,Amazon,amazon.es,10.0,10.0,"Era para mi hija mayor de 19 aÃ±os, palabras d...",Lourdes,Samsung Galaxy A5 - Smartphone libre Android (...,6757,96628
238807,/cellphones/samsung-galaxy-s3-mini/,3/15/2013,ru,ru,Ð¡Ð²ÑÐ·Ð½Ð¾Ð¹,svyaznoy.ru,9.0,10.0,ÐÑÐµÐ³Ð´Ð° Ð¾ÑÐ½Ð¾ÑÐ¸Ð»ÑÑ Ðº ÐÐ½Ð´ÑÐ¾Ð...,ÐÐ»ÐµÐºÑÐµÐ¹,Samsung I8190 Galaxy S III mini 8GB (ÑÐµÑÐ½Ñ...,29037,471
82401,/cellphones/samsung-galaxy-s6-edge-sm-g925f/,9/16/2016,pt,br,Cissa Magazine,cissamagazine.com.br,10.0,10.0,Otimo produto Otimo o produto e a loja tbm che...,Jhennifer alves dos santos figueira,Smartphone Samsung Galaxy S6 Edge 64GB G925I D...,2842,14016


In [50]:
phoneReview_SVD=phoneReviewSampleSVD[['#authorId','#prodID','score']]

In [51]:
#Final Dataset with Author Id, product Id and score
phoneReview_SVD

,#authorId,#prodID,score
278070,425402,30890,10.0
269870,652462,47824,10.0
318920,680724,49954,8.0
170036,593301,43829,2.0
179655,40164,6064,8.0
...,...,...,...
85639,316954,21636,6.0
198745,96628,6757,10.0
238807,471,29037,9.0
82401,14016,2842,10.0


In [52]:
#Read the above dataset using Surprise Dataset 
reader=Reader(rating_scale=(0,10))
phoneReviewBuildSet= Dataset.load_from_df(phoneReview_SVD,reader)

In [53]:
#Build training set
train_set=phoneReviewBuildSet.build_full_trainset()

In [54]:
train_set.ur

defaultdict(list,
            {0: [(0, 10.0)],
             1: [(1, 10.0)],
             2: [(2, 8.0)],
             3: [(3, 2.0)],
             4: [(4, 8.0)],
             5: [(5, 8.0)],
             6: [(6, 10.0)],
             7: [(7, 2.0)],
             8: [(8, 2.0)],
             9: [(9, 10.0)],
             10: [(10, 10.0)],
             11: [(11, 8.0)],
             12: [(12, 10.0), (3054, 10.0)],
             13: [(13, 10.0)],
             14: [(14, 2.0)],
             15: [(15, 4.0)],
             16: [(16, 10.0)],
             17: [(17, 10.0)],
             18: [(18, 10.0)],
             19: [(19, 10.0), (2487, 6.0)],
             20: [(20, 4.0), (2513, 10.0)],
             21: [(21, 10.0)],
             22: [(22, 10.0)],
             23: [(23, 10.0)],
             24: [(24, 6.0)],
             25: [(25, 10.0)],
             26: [(26, 4.0)],
             27: [(27, 10.0)],
             28: [(28, 2.0)],
             29: [(29, 10.0)],
             30: [(30, 10.0)],
             

In [56]:
#Building test set
test_set = train_set.build_anti_testset()

In [57]:
test_set

[(425402, 47824, 8.1186),
 (425402, 49954, 8.1186),
 (425402, 43829, 8.1186),
 (425402, 6064, 8.1186),
 (425402, 39959, 8.1186),
 (425402, 41704, 8.1186),
 (425402, 12978, 8.1186),
 (425402, 21968, 8.1186),
 (425402, 46079, 8.1186),
 (425402, 5481, 8.1186),
 (425402, 6903, 8.1186),
 (425402, 29206, 8.1186),
 (425402, 33724, 8.1186),
 (425402, 35027, 8.1186),
 (425402, 48333, 8.1186),
 (425402, 25778, 8.1186),
 (425402, 7137, 8.1186),
 (425402, 7028, 8.1186),
 (425402, 37399, 8.1186),
 (425402, 31797, 8.1186),
 (425402, 49092, 8.1186),
 (425402, 16146, 8.1186),
 (425402, 47753, 8.1186),
 (425402, 42114, 8.1186),
 (425402, 26421, 8.1186),
 (425402, 49103, 8.1186),
 (425402, 34392, 8.1186),
 (425402, 30523, 8.1186),
 (425402, 29309, 8.1186),
 (425402, 47975, 8.1186),
 (425402, 32259, 8.1186),
 (425402, 14740, 8.1186),
 (425402, 26915, 8.1186),
 (425402, 21407, 8.1186),
 (425402, 44587, 8.1186),
 (425402, 41576, 8.1186),
 (425402, 41721, 8.1186),
 (425402, 27243, 8.1186),
 (425402, 12, 8.1

### Prediction and evaluation using SVD model

In [260]:
svdModel=SVD()
svdModel.fit(train_set)

#### Predicting scores for test users

In [58]:
#predicting scores for test users for products
prediction = svdModel.test(test_set)

#### Evaluation of SVD model using RMSE and MAE

In [59]:
#Evaluation of SVD model using RMSE and MAE
print("RMSE for SVD :",accuracy.rmse(prediction,verbose=True))
print("MAE for SVD :",accuracy.mae(prediction,verbose=True))

RMSE: 0.3509
RMSE for SVD : 0.35088554598595456
MAE:  0.2838
MAE for SVD : 0.2837543217820148


#### Cross validation and evaluation

In [257]:

SVDPredCV =  cross_validate(svdModel,phoneReviewBuildSet,measures=['RMSE', 'MAE'],cv=10)

In [258]:
SVDPredCV

{'test_rmse': array([2.74992647, 2.55666246, 2.74699194, 2.60394958, 2.64396012,
        2.56437131, 2.66841907, 2.74738652, 2.6766713 , 2.5656338 ]),
 'test_mae': array([2.20112968, 2.02894416, 2.14554761, 2.07059025, 2.09073407,
        2.01925696, 2.08843685, 2.16922453, 2.06922932, 2.03074059]),
 'fit_time': (0.7306454181671143,
  0.6966884136199951,
  0.70068359375,
  0.7056138515472412,
  0.6983029842376709,
  0.6628315448760986,
  0.682849645614624,
  0.7855596542358398,
  0.7157754898071289,
  0.6967871189117432),
 'test_time': (0.018850326538085938,
  0.0034813880920410156,
  0.011282205581665039,
  0.007552146911621094,
  0.003905773162841797,
  0.02082991600036621,
  0.007562875747680664,
  0.0,
  0.0091705322265625,
  0.00951242446899414)}

In [259]:
#Evaluation of  Item-Item CF model using CV
print("Mean RMSE for SVD model using CV :",np.mean(list(SVDPredCV['test_rmse'])))
print("Mean MAE for SVD model using CV :",np.mean(list(SVDPredCV['test_mae'])))

Mean RMSE for SVD model using CV : 2.65239725730716
Mean MAE for SVD model using CV : 2.0913834030780096


### Building CF model using KNNWithMeans

#### Item-Item based

In [94]:

knnModel = KNNWithMeans(k=40, sim_options={'name': 'pearson_baseline', 'user_based': False})
knnModel.fit(train_set)


Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.


#### Predicting scores for test users for products

In [95]:
knnPred = knnModel.test(test_set)

In [96]:
knnPred

[Prediction(uid=425402, iid=47824, r_ui=8.1186, est=10, details={'actual_k': 0, 'was_impossible': False}),
 Prediction(uid=425402, iid=49954, r_ui=8.1186, est=8.0, details={'actual_k': 0, 'was_impossible': False}),
 Prediction(uid=425402, iid=43829, r_ui=8.1186, est=2.0, details={'actual_k': 0, 'was_impossible': False}),
 Prediction(uid=425402, iid=6064, r_ui=8.1186, est=8.285714285714286, details={'actual_k': 0, 'was_impossible': False}),
 Prediction(uid=425402, iid=39959, r_ui=8.1186, est=9.0, details={'actual_k': 0, 'was_impossible': False}),
 Prediction(uid=425402, iid=41704, r_ui=8.1186, est=10, details={'actual_k': 0, 'was_impossible': False}),
 Prediction(uid=425402, iid=12978, r_ui=8.1186, est=2.0, details={'actual_k': 0, 'was_impossible': False}),
 Prediction(uid=425402, iid=21968, r_ui=8.1186, est=2.0, details={'actual_k': 0, 'was_impossible': False}),
 Prediction(uid=425402, iid=46079, r_ui=8.1186, est=10, details={'actual_k': 0, 'was_impossible': False}),
 Prediction(uid=42

#### Evaluating KNNModelWith means Item-Item CF model using RMSE and MAE

In [97]:
#Evaluating KNNModelWith means Item-Item CF model using RMSE and MAE
print("RMSE for Item-Item KNN CF model:",accuracy.rmse(knnPred,verbose=True))
print("MAE for Item-Item KNN CF model:",accuracy.mae(knnPred,verbose=True))

RMSE: 2.5612
RMSE for Item-Item KNN CF model: 2.5611557820693966
MAE:  2.0108
MAE for Item-Item KNN CF model: 2.010781059775938


#### Perform cross validation for  Item-Item CF model

In [117]:

perf = cross_validate(knnModel,phoneReviewBuildSet,measures=['RMSE', 'MAE'],cv=10)

Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline si

In [118]:
perf

{'test_rmse': array([2.7705746 , 2.69598965, 2.59248786, 2.58772511, 2.56553132,
        2.69359543, 2.76214805, 2.7893733 , 2.63457038, 2.80977378]),
 'test_mae': array([2.1524085 , 2.09819541, 2.02433749, 2.06533162, 2.0212687 ,
        2.08227554, 2.09429582, 2.14744332, 2.01811335, 2.18224828]),
 'fit_time': (0.4265103340148926,
  0.36659884452819824,
  0.3901851177215576,
  0.3692030906677246,
  0.35692667961120605,
  0.35532355308532715,
  0.4173119068145752,
  0.3973987102508545,
  0.45978617668151855,
  0.3797037601470947),
 'test_time': (0.011008739471435547,
  0.008975744247436523,
  0.013992071151733398,
  0.011955738067626953,
  0.011973381042480469,
  0.01703786849975586,
  0.013961076736450195,
  0.020934104919433594,
  0.010971546173095703,
  0.009974479675292969)}

#### Evaluation of  Item-Item CF model using CV

In [119]:

print("Mean RMSE for item-item CF model using CV :",np.mean(list(perf['test_rmse'])))
print("Mean MAE for item-item CF model using CV :",np.mean(list(perf['test_mae'])))

Mean RMSE for item-item CF model using CV : 2.69017694702132
Mean MAE for item-item CF model using CV : 2.0885918037840105


### Build User-User KNNwithMeans CF model

In [206]:
knnModelUser = KNNWithMeans(k=40, sim_options={'name': 'pearson_baseline', 'user_based': True})
knnModelUser.fit(train_set)
knnPredUser = knnModelUser.test(test_set)

Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.


#### Evaluating KNNModelWith means user-user CF model using RMSE and MAE

In [218]:
print("RMSE for user-user KNN CF model:",accuracy.rmse(knnPredUser))
print("MAE for user-user KNN CF model:",accuracy.mae(knnPredUser))

RMSE: 2.5943
RMSE for user-user KNN CF model: 2.5943154128633656
MAE:  2.0397
MAE for user-user KNN CF model: 2.039687990173788


#### Cross validation for user-user CF model

In [221]:

knnpredUserCV= cross_validate(knnModelUser,phoneReviewBuildSet,measures=['RMSE', 'MAE'],cv=2)
print("Mean RMSE for user-user CF model using CV :",np.mean(list(knnpredUserCV['test_rmse'])))
print("Mean MAE for user-userCF model using CV :",np.mean(list(knnpredUserCV['test_mae'])))

Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Mean RMSE for user-user CF model using CV : 2.6831365868842845
Mean MAE for user-userCF model using CV : 2.0979769251033886


#### Custom function to get top-n recommendations for a user based on rating predictions estimated by SVD and CF models. This function will take prediction  values  and compare rating for each prediction. It will return the user Ids and products from  highest to  lowest bsaed on scores.

In [294]:

def get_top_n_recommendations(predictions, n=5, minimumRating=8.0):
    userRecommend =defaultdict(list)
    for userID, productId, actualRating, estimatedRating, _ in predictions:
        if (estimatedRating >= minimumRating):
               userRecommend[userID].append((productId,estimatedRating))
   # top5recommend= pd.Dataframe(userRecommend.groupby('userId')['productId','score'].sort_values(by='score',ascending=False))

    for uid, user_ratings in userRecommend.items():
        user_ratings.sort(key=lambda x: x[1], reverse=True)
        userRecommend[uid] = user_ratings[:n]
    #get_recommend_userId(userRecommend)
    return userRecommend

#### Custom function that will take an userId and all recommended products from previous function result. It will then display the product names recommended for that user.

In [339]:
#Custom function to 
def get_recommend_userId (getRecommend,userId):
    finalUserRecommend = defaultdict(list)
    prodNameList=[]
    prodNames=[]
    prodNameDF=pd.DataFrame()
    for user, prodRating in getRecommend.items():
        if user==userId:
            finalUserRecommend[user].append(prodRating)
            author_name = (authorReviewNames[authorReviewNames['authorId']==userId]['authorName'])
            
            size =len(prodRating)
           
            for i in range(0,size):
                prodNameList.append(prodRating[i][0])
               # print(prodNameList)
            
            for names in prodNameList:
                
                prodNames.append(phoneReviewProdName[phoneReviewProdName['prodId']==names]['prodNames'])
                
            
       
            
    print("Author:",author_name)
  #  print("Products:",prodNames)         
            
    print("Product recommendations for user {} are {}:".format(author_name,prodNames))
    #return finalUserRecommend

### Recommend top 5 products for test users using SVD and KNNWithMeans

#### Return top 5 recommendation for test users for predictions obtained from SVD model

In [296]:
getRecommend= get_top_n_recommendations(prediction,5)

In [145]:
#list of userIDs and productIDs
getRecommend

defaultdict(list,
            {425402: [(32424, 9.627125563157222),
              (296, 9.231864265924855),
              (25778, 9.112326675422453),
              (46837, 9.096048229479598),
              (2658, 9.006546353221713)],
             652462: [(32424, 9.409898676456978),
              (46837, 9.157072481524098),
              (41819, 9.057109275916732),
              (771, 9.026952631426523),
              (296, 9.014057295158265)],
             680724: [(32424, 9.345068987522765),
              (771, 9.00123792268361),
              (46837, 8.997306964681632),
              (12396, 8.886625713290371),
              (296, 8.878207336077162)],
             593301: [(32424, 8.678384140455316),
              (46837, 8.421719534931018),
              (2824, 8.398438199499902),
              (4410, 8.351007111637568),
              (3572, 8.3376035727693)],
             40164: [(32424, 9.275475418821687),
              (46837, 8.903053075312211),
              (25778, 8.86627350

In [340]:
get_recommend_userId(getRecommend,680724)

Author: 680724    bigvoron
Name: authorName, dtype: object
Product recommendations for user 680724    bigvoron
Name: authorName, dtype: object are [32424    Samsung Galaxy Express I8730
Name: prodNames, dtype: object, 771    Samsung Galaxy S7 32GB (Verizon)
Name: prodNames, dtype: object, 46837    Nokia 5130 XpressMusic
Name: prodNames, dtype: object, 12396    Motorola Moto G LTE- Factory Unlocked US Warra...
Name: prodNames, dtype: object, 296    Samsung Galaxy S7 edge Smartphone, 13,9 cm (5,...
Name: prodNames, dtype: object]:


#### Return top 5 recommendation for test users for predictions obtained from Item based CF model

In [202]:
getKnnRecommend = get_top_n_recommendations(knnPred,n=5)

In [223]:
#test recommendations for a user
get_recommend_userId(getKnnRecommend,680724)

Author: 680724    bigvoron
Name: authorName, dtype: object
Products: [30890    Lenovo ZUK Z1 4G Smartphone 5.5" Fingerprint I...
Name: prodNames, dtype: object, 47824    Nokia E75 Smartphone (UMTS, GPS, UKW Radio, 3 ...
Name: prodNames, dtype: object, 41704    Samsung (936) S3850 Corby II
Name: prodNames, dtype: object, 46079    Samsung La Fleur GT-S5230
Name: prodNames, dtype: object, 5481    LG G5 Smartphone da 32 GB, 5.3" QHD, 4G LTE, F...
Name: prodNames, dtype: object]


defaultdict(list,
            {680724: [[(30890, 10),
               (47824, 10),
               (41704, 10),
               (46079, 10),
               (5481, 10)]]})

#### Return top 5 recommendation for test users for predictions obtained from User based CF model

In [224]:
knnUserRecommend= get_top_n_recommendations(knnPredUser,n=5)
get_recommend_userId(knnUserRecommend,680724)

Author: 680724    bigvoron
Name: authorName, dtype: object
Products: [30890    Lenovo ZUK Z1 4G Smartphone 5.5" Fingerprint I...
Name: prodNames, dtype: object, 47824    Nokia E75 Smartphone (UMTS, GPS, UKW Radio, 3 ...
Name: prodNames, dtype: object, 43829    Apple iPhone 3Gs 16GB black
Name: prodNames, dtype: object, 6064    Huawei P8lite zwart / 16 GB
Name: prodNames, dtype: object, 39959    HTC 7 Trophy
Name: prodNames, dtype: object]


defaultdict(list,
            {680724: [[(30890, 8.0),
               (47824, 8.0),
               (43829, 8.0),
               (6064, 8.0),
               (39959, 8.0)]]})

### Finding and Inferences

#### Results for test set 
| Model/metrics|RMSE|MAE|RMSE with CV|MAE with CV|
|:---:|:-------------|:-----------|:-----------|:-----------|
| SVD|0.35|0.28|2.65|2.01|
| KNNWithMeans Item Based|2.56|2.01|2.69|2.08|
| KNNWithMeans User Based|2.59|2.04|2.68|2.09|



From the results above, SVD has the least MAE and RMSE values. This is because it has reduced sparsity factor than KNNWithmeans due to decomposition. 

For KNNWithMeans,  MAE and RMSE scores, with and without cross validation, are almost same. It was observed that it takes more computation time for prediction of ratings in comparison to SVD algorithm.

### USE CASES

#### Popularity Based Recommendation System
 
Popularity based recommendation system is used in cases where personalization of items  are not required such as News.In this case, the highly trending news,videos, movies etc. rated by other users will be displayed on the screen.


There are cases when a user is new to the website and his preference characterisitics are unknown to the system. In this case we can use popularity based system to make recommendations that popular among other users.

Examples:
1. Google News
2. YouTube trending videos
3. Suggestions on Amazon website home page when customer is not logged in

#### CF based recommendation system
This recommendation system provides more personalized experience to the user.  In order to provide more personlised suggestions to user, similarity between users is calculated based on the ratings provided in history or between the items the users have provided have rated for.

CF provides users range of diversity in the options keeping in the mind the interests of the user.

Examples:
1. Netflix
2. Amazon
3. Prime video
4. Myntra

#### Methods that can help improve suggestions for users
Each of the recommendation systems have their own benefits and disadvantages. The suggestions would be more variant and relevant if we have Hybrid recommendation system that will combine pros of 2 or more systems.

For example, Content based recommendation system can be used to retrieve description of item or movies. And if we use this along with CF item-based model, we can get variants of the product along with similar product suggestions.

